In [ ]:
import chess
import chess.engine
import sqlite3
import pandas as pd
from pathlib import Path
import json
import re

class ChessGameAnalyzer:
    def __init__(self, stockfish_path: str = "stockfish"):
        self.engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
        self.engine.configure({"Threads": 4, "Hash": 256})
        print("Initializing Stockfish...")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.engine.quit()

    def get_position_eval(self, board: chess.Board) -> float:
        """Get static evaluation of position"""
        result = self.engine.analyse(board, chess.engine.Limit(depth=23))
        return result['score'].relative.score(mate_score=10000)

    def analyze_game(self, moves_str: str) -> dict:
        analysis = {
            'moves': [],
            'white_stats': {
                'total_moves': 0,
                'inaccuracies': 0,
                'mistakes': 0,
                'blunders': 0,
                'total_centipawn_loss': 0,
                'avg_centipawn_loss': 0,
                'accuracy': 0
            },
            'black_stats': {
                'total_moves': 0,
                'inaccuracies': 0,
                'mistakes': 0,
                'blunders': 0,
                'total_centipawn_loss': 0,
                'avg_centipawn_loss': 0,
                'accuracy': 0
            }
        }

        board = chess.Board()
        moves = self.parse_moves(moves_str)
        
        for move_number, (move, color) in enumerate(moves):
            try:
                # Get evaluation before move
                info = self.engine.analyse(board, chess.engine.Limit(depth=18), multipv=2)
                best_score = info[0]['score'].relative.score(mate_score=10000)
                
                # Make the actual move
                board.push(move)
                
                # Get evaluation after move
                after_score = -self.get_position_eval(board)  # Negate because perspective changes
                
                # Calculate centipawn loss
                eval_diff = max(0, best_score - (-after_score))
                
                # Classify move
                stats_key = 'white_stats' if color == chess.WHITE else 'black_stats'
                analysis[stats_key]['total_moves'] += 1
                analysis[stats_key]['total_centipawn_loss'] += eval_diff

                # Lichess-style thresholds
                if eval_diff >= 300:
                    analysis[stats_key]['blunders'] += 1
                elif eval_diff >= 100:
                    analysis[stats_key]['mistakes'] += 1
                elif eval_diff >= 50:
                    analysis[stats_key]['inaccuracies'] += 1
                
            except Exception as e:
                print(f"Error analyzing move {move}: {e}")
                continue

        # Calculate final statistics using Lichess formula
        for color in ['white_stats', 'black_stats']:
            if analysis[color]['total_moves'] > 0:
                avg_loss = analysis[color]['total_centipawn_loss'] / analysis[color]['total_moves']
                analysis[color]['avg_centipawn_loss'] = round(avg_loss)
                # Lichess accuracy formula
                analysis[color]['accuracy'] = round(max(0, min(100, 103.1668 - (avg_loss * 0.667))))

        return analysis

    def parse_moves(self, moves_str: str) -> list:
        """Parse moves string into list of (move, color) tuples"""
        moves = []
        board = chess.Board()
        
        # Clean up the input string
        moves_str = re.sub(r'\{.*?\}', '', moves_str)
        moves_str = re.sub(r'(1-0|0-1|1/2-1/2|\*)', '', moves_str)
        print(f"Moves: {moves_str}")
        
        tokens = moves_str.strip().split()
        i = 0
        while i < len(tokens):
            token = tokens[i]
            
            if '.' in token:
                parts = token.split('.')
                if len(parts) > 1 and parts[1]:
                    white_san = parts[1]
                else:
                    i += 1
                    if i >= len(tokens):
                        break
                    white_san = tokens[i]
                
                try:
                    white_move = board.parse_san(white_san)
                    moves.append((white_move, chess.WHITE))
                    board.push(white_move)
                except Exception as e:
                    print(f"Error parsing white move {white_san}: {e}")
                    break
                
                i += 1
                if i < len(tokens):
                    black_san = tokens[i]
                    if '.' in black_san:
                        i -= 1
                        continue
                    try:
                        black_move = board.parse_san(black_san)
                        moves.append((black_move, chess.BLACK))
                        board.push(black_move)
                    except Exception as e:
                        print(f"Error parsing black move {black_san}: {e}")
                        break
            i += 1
            
        return moves

def main():
    DB_PATH = "../../data/processed/chess_games.db"
    TEST_GAMES = 1
    
    print("Stockfish engine initialized successfully")
    with ChessGameAnalyzer() as analyzer:
        print(f"\nFetching {TEST_GAMES} test games from database...")
        conn = sqlite3.connect(DB_PATH)
        games_df = pd.read_sql(f"""
            SELECT id, white, black, moves
            FROM chess_games
            WHERE moves IS NOT NULL
            AND length(moves) > 10
            LIMIT {TEST_GAMES}
        """, conn)
        conn.close()
        
        for _, row in games_df.iterrows():
            print(f"\nAnalyzing game {row['id']}: {row['white']} vs {row['black']}")
            
            analysis = analyzer.analyze_game(row['moves'])
            
            print("\nWhite's Analysis:")
            white_stats = analysis['white_stats']
            print(f"Inaccuracies: {white_stats['inaccuracies']}")
            print(f"Mistakes: {white_stats['mistakes']}")
            print(f"Blunders: {white_stats['blunders']}")
            print(f"Average centipawn loss: {white_stats['avg_centipawn_loss']}")
            print(f"Accuracy: {white_stats['accuracy']}%")
            
            print("\nBlack's Analysis:")
            black_stats = analysis['black_stats']
            print(f"Inaccuracies: {black_stats['inaccuracies']}")
            print(f"Mistakes: {black_stats['mistakes']}")
            print(f"Blunders: {black_stats['blunders']}")
            print(f"Average centipawn loss: {black_stats['avg_centipawn_loss']}")
            print(f"Accuracy: {black_stats['accuracy']}%")
            
            with open('detailed_analysis.json', 'w') as f:
                json.dump({
                    'game_id': row['id'],
                    'white': row['white'],
                    'black': row['black'],
                    'analysis': analysis
                }, f, indent=2)
            print("\nDetailed analysis saved to detailed_analysis.json")

if __name__ == "__main__":
    main()

In [1]:
from utils import GameAnalyzer

game_moves = """1. Nf3 Nc6 2. d4 d5 3. c4 Nf6 4. cxd5 Nxd5 5. e4 Nf6 6. d5 Nb8 
7. Nc3 e6 8. Bg5 Be7 9. e5 Nxd5 10. Nxd5 Bxg5 11. Nxg5 Qxd5 12. Qxd5 exd5 
13. f4 O-O 14. O-O-O c6 15. Be2 Be6 16. g4 h6 17. Nh3 f5 18. exf6 Bf5 
19. gxf5 Rxf6 20. Bd3 Nd7 21. Rhg1 Nc5 22. Bc2 a5 23. Rg6 Raf8 24. Rdg1 R8f7 
25. Nf2 b5 26. Ng4 Rxg6 27. fxg6 Rxf4 28. Ne5 Rf6 29. Nf7 a4 30. a3 Na6 
31. h4 b4 32. h5 bxa3 33. bxa3 Nc5 34. Ne5 Nb3+ 35. Bxb3 axb3 36. Kb2 Rf5 
37. Rh1 Rxe5 38. Kxb3 Re3+ 39. Kb4 d4 40. a4 d3 41. Kc3 c5 42. Rb1 Kf8 
43. Rb8+ Ke7 44. Rb7+ Kf6 45. Rf7+ Kg5 46. Rxg7 Kxh5 47. Kd2 Re6 48. Kxd3 Rxg6 
49. Rxg6 Kxg6 50. a5 h5 51. a6 h4 52. a7 h3 53. a8=Q h2 54. Qh1 Kf5 55. Qh2 
Ke6 56. Kc4 Kd7 57. Qe5 Kd8 58. Qxc5 Ke8 59. Qa7"""

analyzer = GameAnalyzer()
positions = analyzer.get_positions_from_moves(game_moves)
middlegame_start, endgame_start = analyzer.find_phase_transitions(positions)

print(f"\nGame Phases:")
print(f"Opening: moves 1-{middlegame_start}")
print(f"Middlegame: moves {middlegame_start}-{endgame_start}")
print(f"Endgame: moves {endgame_start}-end")



Game Phases:
Opening: moves 1-11
Middlegame: moves 11-26
Endgame: moves 26-end


In [ ]:
from utils import GameAnalyzer

game_moves = """1. e4 e6 2. Nf3 d5 3. exd5 exd5 4. d4 Nf6 5. Nc3 Bd6 6. Bg5 c6 7. Qd2 Bg4 8. O-O-O O-O 9. Kb1 b5 10. Re1 Nbd7 11. Be2 b4 12. Nd1 a5 13. h3 Bxf3 14. Bxf3 a4 15. c3 bxc3 16. Nxc3 Rb8 17. Nxa4 Ba3 18. Qc3 Bb4 19. Qc2 Bxe1 20. Rxe1 Qa5 21. Re3 Ra8 22. Ra3 Qe1+ 23. Bd1 Ne4 24. Be3 Rfb8 25. f3 Rxa4 26. Qxa4 Nc3+ 27. Rxc3 Qxc3 28. Bc1 Qc4 29. Qa7 Qd3+ 30. Bc2 Qb5 31. Qxd7 h6 32. Qc7 Rb7 33. Qc8#"""

analyzer = GameAnalyzer(depth=20, threads=7, debug=True) 
positions = analyzer.get_positions_from_moves(game_moves)
evaluations = analyzer.evaluate_moves(positions)
phases = analyzer.find_phase_transitions(positions)
analyzer.print_analysis(evaluations, phases)
analyzer.close_engine()


ANALYZING GAME MOVES

Engine Configuration:
- Depth: 22
- Stockfish Version: 17
--------------------------------------------------------------------------------



Analyzing White move Qc8#: 100%|████████████████| 65/65 [05:31<00:00,  5.10s/it]


MOVE ANALYSIS

Detailed Move Analysis:
----------------------------------------------------------------------------------------------------
Move  | Color  |   Move   |  Eval   | Change  |  Quality   |    Best Move   
----------------------------------------------------------------------------------------------------
  1   | White  |    e4    |  +0.3   |  +0.0   |    Good    |                
  1   | Black  |    e6    |  +0.3   |  -0.0   |    Good    |                
  2   | White  |   Nf3    |  +0.1   |  -0.2   |    Good    |                
  2   | Black  |    d5    |  +0.2   |  +0.1   |    Good    |                
  3   | White  |   exd5   |  +0.2   |  -0.0   |    Good    |                
  3   | Black  |   exd5   |  +0.2   |  +0.0   |    Good    |                
  4   | White  |    d4    |  +0.2   |  -0.0   |    Good    |                
  4   | Black  |   Nf6    |  +0.1   |  -0.1   |    Good    |                
  5   | White  |   Nc3    |  +0.0   |  -0.1   |    Good    |     

In [2]:
from utils import GameAnalyzer

game_moves = """1. Nf3 Nc6 2. d4 d5 3. c4 Nf6 4. cxd5 Nxd5 5. e4 Nf6 6. d5 Nb8 
7. Nc3 e6 8. Bg5 Be7 9. e5 Nxd5 10. Nxd5 Bxg5 11. Nxg5 Qxd5 12. Qxd5 exd5 
13. f4 O-O 14. O-O-O c6 15. Be2 Be6 16. g4 h6 17. Nh3 f5 18. exf6 Bf5 
19. gxf5 Rxf6 20. Bd3 Nd7 21. Rhg1 Nc5 22. Bc2 a5 23. Rg6 Raf8 24. Rdg1 R8f7 
25. Nf2 b5 26. Ng4 Rxg6 27. fxg6 Rxf4 28. Ne5 Rf6 29. Nf7 a4 30. a3 Na6 
31. h4 b4 32. h5 bxa3 33. bxa3 Nc5 34. Ne5 Nb3+ 35. Bxb3 axb3 36. Kb2 Rf5 
37. Rh1 Rxe5 38. Kxb3 Re3+ 39. Kb4 d4 40. a4 d3 41. Kc3 c5 42. Rb1 Kf8 
43. Rb8+ Ke7 44. Rb7+ Kf6 45. Rf7+ Kg5 46. Rxg7 Kxh5 47. Kd2 Re6 48. Kxd3 Rxg6 
49. Rxg6 Kxg6 50. a5 h5 51. a6 h4 52. a7 h3 53. a8=Q h2 54. Qh1 Kf5 55. Qh2 
Ke6 56. Kc4 Kd7 57. Qe5 Kd8 58. Qxc5 Ke8 59. Qa7"""

analyzer = GameAnalyzer(depth=16) 
positions = analyzer.get_positions_from_moves(game_moves)
evaluations = analyzer.evaluate_moves(positions)
phases = analyzer.find_phase_transitions(positions)
analyzer.print_analysis(evaluations, phases)
analyzer.close_engine()

Analyzing Black move Nc6:   2%|▎                | 2/117 [00:00<00:37,  3.06it/s]

Depth: 16, Nodes: 150978, Time: 0.324
Using Stockfish version: Stockfish 17



Analyzing White move Qa7: 100%|███████████████| 117/117 [00:11<00:00, 10.23it/s]



Move Analysis:
Move  Color  Move   Eval   Change  Quality
---------------------------------------------
1     White  Nf3    +0.2   +0.0    Good
1     Black  Nc6    +0.6   +0.3    Good
2     White  d4     +0.6   -0.0    Good
2     Black  d5     +0.5   -0.1    Good
3     White  c4     +0.4   -0.1    Good
3     Black  Nf6    +1.5   +1.1    Mistake
4     White  cxd5   +1.5   -0.1    Good
4     Black  Nxd5   +1.5   +0.1    Good
5     White  e4     +1.5   +0.0    Good
5     Black  Nf6    +1.7   +0.1    Good
6     White  d5     +1.6   -0.1    Good
6     Black  Nb8    +1.5   -0.1    Good
7     White  Nc3    +1.6   +0.1    Good
7     Black  e6     +1.4   -0.2    Good
8     White  Bg5    +1.1   -0.3    Good
8     Black  Be7    +1.1   +0.1    Good
9     White  e5     -0.3   -1.5    Mistake
9     Black  Nxd5   -0.5   -0.2    Good
10    White  Nxd5   -0.9   -0.4    Good
10    Black  Bxg5   -1.1   -0.1    Good
11    White  Nxg5   -1.4   -0.3    Good
11    Black  Qxd5   -1.3   +0.0    Good
12    Whi

In [ ]:
from utils import GameAnalyzer

game_moves = """1. Nf3 Nc6 2. d4 d5 3. c4 Nf6 4. cxd5 Nxd5 5. e4 Nf6 6. d5 Nb8 
7. Nc3 e6 8. Bg5 Be7 9. e5 Nxd5 10. Nxd5 Bxg5 11. Nxg5 Qxd5 12. Qxd5 exd5 
13. f4 O-O 14. O-O-O c6 15. Be2 Be6 16. g4 h6 17. Nh3 f5 18. exf6 Bf5 
19. gxf5 Rxf6 20. Bd3 Nd7 21. Rhg1 Nc5 22. Bc2 a5 23. Rg6 Raf8 24. Rdg1 R8f7 
25. Nf2 b5 26. Ng4 Rxg6 27. fxg6 Rxf4 28. Ne5 Rf6 29. Nf7 a4 30. a3 Na6 
31. h4 b4 32. h5 bxa3 33. bxa3 Nc5 34. Ne5 Nb3+ 35. Bxb3 axb3 36. Kb2 Rf5 
37. Rh1 Rxe5 38. Kxb3 Re3+ 39. Kb4 d4 40. a4 d3 41. Kc3 c5 42. Rb1 Kf8 
43. Rb8+ Ke7 44. Rb7+ Kf6 45. Rf7+ Kg5 46. Rxg7 Kxh5 47. Kd2 Re6 48. Kxd3 Rxg6 
49. Rxg6 Kxg6 50. a5 h5 51. a6 h4 52. a7 h3 53. a8=Q h2 54. Qh1 Kf5 55. Qh2 
Ke6 56. Kc4 Kd7 57. Qe5 Kd8 58. Qxc5 Ke8 59. Qa7"""

analyzer = GameAnalyzer(depth=25) 
positions = analyzer.get_positions_from_moves(game_moves)
evaluations = analyzer.evaluate_moves(positions)
phases = analyzer.find_phase_transitions(positions)
analyzer.print_analysis(evaluations, phases)
analyzer.close_engine()